In [2]:
# uncomment the line below to download ipywidgets, if you don't have it
# !pip install ipywidgets

In [3]:
import ipywidgets as wid
from pylgbst.hub import MoveHub
from pylgbst import get_connection_bleak
import time
import random

In [4]:
# initializing the controller
gamepad = wid.Controller()

In [ ]:
# connecting to the MoveHub using its individual Bluetooth address 
conn = get_connection_bleak(hub_mac='00:16:53:C3:C2:4F', hub_name=MoveHub.DEFAULT_NAME)
hub = MoveHub(conn)

In [ ]:
def on_sent(data):
    if gamepad.connected:
        # moving forward (right joystick)
        if data['owner'].axes[3].value < 0:
            hub.motor_AB.timed(1, 0.1, 0.1)
            
        # moving backwards (right joystick)
        if data['owner'].axes[3].value > 0:
            hub.motor_AB.timed(1, -0.1, -0.1)
            
        # moving to the right (right joystick)
        if data['owner'].axes[2].value < 0:
            hub.motor_A.angled(540, 1)
            
        # moving to the left (right joystick)
        if data['owner'].axes[2].value > 0:
            hub.motor_B.angled(540, 1)
            
        # speeding forward (R2 button)
        if data['owner'].button[7].value:
            hub.motor_AB.timed(1, 2, 2)
        
        # stoping the motors fully (R1 button)
        if data['owner'].button[5].value:
            hub.motor_external.stop()
        
        # changing led color (select button)
        if data['owner'].button[8].value:
            # randomly generated color
            color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
            hub.led.set_color(color)
            
        # move in a square (B button)
        if data['owner'].button[0].value:
            for count in range(4):
                hub.motor_AB.timed(2, 0.2, 0.2)
                hub.motor_A.angled(540, 1)
        
        # move in a circle (A button)
        if data['owner'].button[1].value:
            for count in range(4):
                hub.motor_AB.timed(1.1, (-0.5), (-0.9))
        
        # rotate 360° (Y button)
        if data['owner'].button[2].value:
            hub.motor_A.timed(1.7, 1)
            
        # distance detection (up arrow) 
        if data['owner'].button[12].value:
            def callback(distance):
                print("Distance: %s" % (distance))
                
            hub.vision_sensor.subscribe(callback, mode=VisionSensor.DISTANCE_INCHES)
            time.sleep(2)# play with sensor while it waits
            hub.vision_sensor.unsubscribe(callback)
            
        # color detection (left arrow) 
        if data['owner'].button[14].value:
            def callback(color):
                print("Color: %s" % (color))
                
            hub.vision_sensor.subscribe(callback, mode=VisionSensor.COLOR_INDEX)
            time.sleep(2)# play with sensor while it waits
            hub.vision_sensor.unsubscribe(callback)
            
        # detect color and distance (down arrow)
        if data['owner'].button[13].value:
            def callback(color, distance):
                print("Color: %s / Distance: %s" % (color, distance))
                
            hub.vision_sensor.subscribe(callback, mode=VisionSensor.COLOR_DISTANCE_FLOAT)
            time.sleep(2)# play with sensor while it waits
            hub.vision_sensor.unsubscribe(callback)
            
        # detect 3-axis position (right arrow)
        if data['owner'].button[15].value:
            def callback(roll, pitch, yaw):
                print("Roll: %s / Pitch: %s / Yaw: %s" % (roll, pitch, yaw))
                
            hub.tilt_sensor.subscribe(callback, mode=TiltSensor.MODE_2AXIS_ACCEL)
            time.sleep(2)# play with sensor while it waits
            hub.tilt_sensor.unsubscribe(callback)

        # second joystick for controlling head position? (experiment)
        
gamepad.observe(on_sent)
gamepad
# buttons are corresponding to a 8BitDo Controller

In [ ]:
# disconnecting from the MoveHub
hub.disconnect()